<a href="https://colab.research.google.com/github/BenWheeler99/Capstone-Project/blob/main/Book_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This line of code imports the datasets library and the langdetect library.
!pip install datasets
!pip install langdetect
!pip install faiss-cpu

#This line of code import the drive library from the google.colab library
from google.colab import drive
import pandas as pd
from datasets import load_dataset
from langdetect import detect
from transformers import T5Tokenizer
import random
from sentence_transformers import SentenceTransformer
import faiss

#This line of code uses the mount method from the drive method to mount my google drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset. This line of code gathers the "BrightData/Goodreads-Books" dataset form HuggingFace, it internally converts it to datasets format from csv
dataset = load_dataset("BrightData/Goodreads-Books", split="train")

# Shuffle and select 1,000,000 random samples, you can replicate the random by using the seed=42 parameter
sampled_dataset = dataset.shuffle(seed=42).select(range(50_000))

# Verify the size
print(sampled_dataset)
len(sampled_dataset)


In [ ]:
dataset = sampled_dataset.filter(lambda x: x["summary"] is not None and x["summary"] != "")

# 🔹 List of columns to REMOVE (Change these to your needs)
columns_to_remove = ["url", "star_rating", "num_ratings", "num_reviews", "about_author","community_reviews", "kindle_price"]

# 🛠 Drop the columns
dataset = dataset.remove_columns(columns_to_remove)
# `errors="ignore"` prevents crashing if a column doesn't exist

# Function to check if the summary is in English
def is_english(summary):
    try:
        return detect(summary) == "en"  # Detects language and keeps only English
    except:
        return False  # If detection fails, assume it's not English

# Apply the filter
dataset = dataset.filter(lambda x: is_english(x["summary"]))

# Define the save path (change this if needed)
save_path = "/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv"

# Save as CSV
dataset.to_csv(save_path, index=False)  # index=False avoids adding an extra column

print(f"Dataset saved to: {save_path}")

len(dataset)

In [ ]:
#Start here unless you want to manipulate the base data.
from datasets import load_dataset
from google.colab import drive
import pandas as pd
from langdetect import detect
from transformers import T5Tokenizer
import random
from sentence_transformers import SentenceTransformer
import faiss

# Load the dataset (replace 'your_dataset.csv' with the correct path)
dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv'}, split='train')


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Function to generate flexible prompts
def generate_prompt(summary):
    prompts = [
        f"Recommend me a book based on this description: {summary}",
        f"Here is a book summary: {summary}. What would be a good book to match?",
        f"Find a book that fits this summary: {summary}",
        f"Here’s a brief description, what book matches this: {summary}"
    ]

    # Randomly select one of the prompts
    return random.choice(prompts)

# Apply the tokenization function to the dataset
def tokenize_data(examples, batched=True):
    if batched:
        # For batched processing, ensure we process multiple examples at once
        input_texts = [generate_prompt(summary) for summary in examples['summary']]
        output_texts = examples['name']

        # Tokenize input and output with max_length set explicitly to 128
        input_encodings = tokenizer(input_texts, truncation=True, padding='max_length', max_length=128, return_tensors="pt")
        output_encodings = tokenizer(output_texts, truncation=True, padding='max_length', max_length=32, return_tensors="pt")

        # Ensure all inputs have consistent lengths and include attention_mask
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': output_encodings['input_ids'],
        }
    else:
        # For single example processing
        input_text = generate_prompt(examples['summary'])
        output_text = examples['name']

        # Tokenize with max_length explicitly set
        input_encoding = tokenizer(input_text, truncation=True, padding='max_length', max_length=128, return_tensors="pt")
        output_encoding = tokenizer(output_text, truncation=True, padding='max_length', max_length=32, return_tensors="pt")

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': output_encoding['input_ids'].flatten(),
        }

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Apply the tokenization function to the dataset
tokenized_dataset = dataset.map(tokenize_data, batched=True)

dataset = tokenized_dataset

# Define the save path (change this if needed)
save_path = "/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv"

# Save as CSV
dataset.to_csv(save_path, index=False)  # index=False avoids adding an extra column

print(f"Dataset saved to: {save_path}")

Map:   0%|          | 0/36329 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/37 [00:00<?, ?ba/s]

Dataset saved to: /content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv


In [ ]:
len(dataset)

36329

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

# Load dataset (assuming it's a CSV)
dataset = pd.read_csv("/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv")

# Use SentenceTransformer to create embeddings of the summaries
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
summary_embeddings = embedding_model.encode(dataset["summary"].tolist(), convert_to_tensor=True)

# Store embeddings in FAISS index
index = faiss.IndexFlatL2(summary_embeddings.shape[1])
index.add(summary_embeddings.cpu().numpy())  # FAISS index


In [ ]:
from torch.utils.data import DataLoader

# Convert the tokenized dataset to a DataLoader
train_dataloader = DataLoader(tokenized_dataset, batch_size=8, shuffle=True)


In [ ]:
from transformers import T5ForConditionalGeneration

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-base')


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',              # Output directory for model checkpoints
    evaluation_strategy="epoch",         # Evaluate every epoch (this matches save strategy)
    save_strategy="epoch",               # Save model at the end of each epoch
    learning_rate=5e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size per device
    per_device_eval_batch_size=8,        # Batch size for evaluation
    num_train_epochs=3,                  # Number of training epochs
    weight_decay=0.01,                   # Weight decay for optimization
    logging_dir='./logs',                # Directory to save logs
    logging_steps=10,                    # Log every 10 steps
    save_steps=500,                      # Save the model every 500 steps (this will be ignored with `save_strategy="epoch"`)
    save_total_limit=3,                  # Limit the total number of saved checkpoints
    load_best_model_at_end=True,         # Load the best model at the end of training
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                   # The model to train
    args=training_args,             # Training arguments
    train_dataset=tokenized_dataset,  # Training dataset
    eval_dataset=tokenized_dataset,   # Evaluation dataset (you can use a separate validation set if available)
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# ✅ Define save path
save_path = "/content/drive/MyDrive/Final/book-recommender-model-v3"

# ✅ Save model weights
model.save_pretrained(save_path)

# ✅ Save tokenizer
tokenizer.save_pretrained(save_path)

print(f"✅ Model and tokenizer saved to {save_path}")


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/Final/book-recommender-model-v3"  # Change this to your saved model path
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def hybrid_recommendation(prompt, top_k=1):
    # Step 1️⃣: Convert prompt to embedding
    prompt_embedding = embedding_model.encode([prompt], convert_to_tensor=True).cpu().numpy()

    # Step 2️⃣: Retrieve closest book summaries using FAISS
    _, indices = index.search(prompt_embedding, top_k)
    matched_books = dataset.iloc[indices[0]]  # Retrieve matched book(s)

    # Step 3️⃣: Generate prompt for LLM based on retrieved books
    matched_titles = matched_books['name'].tolist()
    refined_prompt = f"Based on these books: {', '.join(matched_titles)}, recommend a book for the following prompt: {prompt}"

    # Step 4️⃣: Generate output using fine-tuned LLM
    inputs = tokenizer(refined_prompt, return_tensors="pt", truncation=True, max_length=128).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=32)

    # Step 5️⃣: Decode & return the book title
    recommended_book = tokenizer.decode(output[0], skip_special_tokens=True)
    return recommended_book, matched_books[["name", "summary"]]

# 🔥 Test the system
prompt = "Recommend me a book about basketball."
book_title, retrieved_books = hybrid_recommendation(prompt)
print(f"📚 **Recommended Book:** {book_title}")
print("\n🔎 **Closest Matches in Dataset:**")
print(retrieved_books)


📚 **Recommended Book:** Basketball: How to Play Post ASAP In Basketball: Scoring AGAINST TALLER DEFENDERS MADE EASY

🔎 **Closest Matches in Dataset:**
                                                    name  \
19979  LEARN HOW TO PLAY POST ASAP IN BASKETBALL: SCO...   

                                                 summary  
19979  The primary purpose of this book is to teach y...  
